# DATA AUGMENTATION

Due to the poor size of validation dataset (~10), we moved validation dataset to the training dataset and we will pick ~10-20% of the training dataset as validation data

We don't have much data to create a better model. With the utility of data augmentation, we generate more images. (Only for Training and validation  data)

data size:
- NORMAL: 1341 * 2 = 2682
- PNEUMONIA: 3875 *2 = 7750
total = 10468

In [1]:
import os
import cv2
import numpy as np
import scipy
from keras.preprocessing.image import ImageDataGenerator
import chardet


# Specify the path to the folder containing the images
train_dir = '../data/chest_Xray/train'

# Define the number of augmented images to generate for each class
num_aug_normal = 1341
num_aug_pneumonia = 3875

# Create separate subfolders for each class of augmented images
aug_dir_normal = '../data/chest_Xray/train_augmented/NORMAL'
aug_dir_pneumonia = '../data/chest_Xray/train_augmented/PNEUMONIA'
if not os.path.exists(aug_dir_normal):
    os.makedirs(aug_dir_normal)

if not os.path.exists(aug_dir_pneumonia):
    os.makedirs(aug_dir_pneumonia)

# Create an instance of ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,      # Rotate images by 20 degrees
    zoom_range=0.2,         # Zoom in/out images by 20%
    width_shift_range=0.1,  # Shift the width of the image by 10%
    height_shift_range=0.1, # Shift the height of the image by 10%
    horizontal_flip=True,   # Flip images horizontally
    vertical_flip=True      # Flip images vertically
)

aug_iter = datagen.flow_from_directory(
    train_dir,
    classes=['NORMAL', 'PNEUMONIA'],
    target_size=(224, 224),  # Resize images to 224x224 pixels
    batch_size=32,           # Generate images in batches of 32
    # save_to_dir='augmented_images/', # Save the augmented images to this directory
    save_prefix='aug_',      # Add a prefix to the filenames of the augmented images
    class_mode='categorical',
    save_format='jpg'        # Save the augmented images in JPEG format
)

num_aug_normal_generated = 0
num_aug_pneumonia_generated = 0

# Loop through each batch of augmented images and save them to the appropriate subfolder
for i, (images, labels) in enumerate(aug_iter):
    for j, label in enumerate(labels):
        if np.argmax(label) == 0: # If the label corresponds to the 'normal' class
            if num_aug_normal_generated < num_aug_normal:
                filename = os.path.join(aug_dir_normal, f"aug_{num_aug_normal_generated}.jpg")
                cv2.imwrite(filename, images[j])
                num_aug_normal_generated += 1
        else: # If the label corresponds to the 'pneumonia' class
            if num_aug_pneumonia_generated < num_aug_pneumonia:
                filename = os.path.join(aug_dir_pneumonia, f"aug_{num_aug_pneumonia_generated}.jpg")
                cv2.imwrite(filename, images[j])
                num_aug_pneumonia_generated += 1
        
        if num_aug_normal_generated == num_aug_normal and num_aug_pneumonia_generated == num_aug_pneumonia:
            break
        
    if num_aug_normal_generated == num_aug_normal and num_aug_pneumonia_generated == num_aug_pneumonia:
        break


Found 5216 images belonging to 2 classes.
